In [ ]:
%%capture
# Run preprocess_models.ipynb to create preprocessed data (if not already done)
%run ../preprocess_models.ipynb

# Mixture Model

This notebook continues from `models.ipynb` and applies a mixture model approach to classify insurance claims.

**Prerequisites:** Run `models.ipynb` first to create the preprocessed data and train/test splits.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np

# Load the preprocessed data from preprocess_models.ipynb
# All data is saved in data/processed/ after running preprocess_models.ipynb

df = pd.read_csv('../../data/processed/df_preprocessed.csv')
X_train = pd.read_csv('../../data/processed/X_train.csv')
X_test = pd.read_csv('../../data/processed/X_test.csv')
y_train = pd.read_csv('../../data/processed/y_train.csv').squeeze()
y_test = pd.read_csv('../../data/processed/y_test.csv').squeeze()

print("✓ Loaded preprocessed data from data/processed/")
print(f"\nData shape: {df.shape}")
print(f"Train set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\n✓ Ready for mixture model!")

✓ Loaded preprocessed data from data/processed/

Data shape: (9912, 18)
Train set: 7929 samples, 17 features
Test set: 1983 samples

✓ Ready for mixture model!


## Step 1: Prepare Data for Mixture Model

We need to encode all categorical variables and scale numerical features for the GMM to work properly.

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import numpy as np

# Encode categorical variables
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# print(X_train_encoded.columns)
# print(X_test_encoded.columns)

# Ensure train and test have the same columns (in case of missing categories)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Standardize features (important for GMM distance calculations)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

print(f"Encoded feature space: {X_train_encoded.shape[1]} features")
print(f"Training set: {X_train_scaled.shape[0]} samples")
print(f"Test set: {X_test_scaled.shape[0]} samples")

Encoded feature space: 24 features
Training set: 7929 samples
Test set: 1983 samples


## Step 2: Fit Gaussian Mixture Model

We'll start with K=3 components to discover latent risk groups in our population. The GMM will cluster customers based on their feature patterns.

In [6]:
# Fit Gaussian Mixture Model with K=3 components
n_components = 3
gmm = GaussianMixture(
    n_components=n_components,
    covariance_type='full',  # Allow each component its own full covariance matrix
    random_state=42,
    n_init=10  # Multiple initializations for stability
)

gmm.fit(X_train_scaled)

# Get component assignments for training data
train_component_probs = gmm.predict_proba(X_train_scaled)  # Posterior probabilities
train_components = gmm.predict(X_train_scaled)  # Hard assignments

# print(train_component_probs[17:20])
# print(train_components[17:20])

print(f"Fitted GMM with {n_components} components")
print(f"\nComponent sizes in training set:")
for k in range(n_components):
    count = np.sum(train_components == k)
    pct = 100 * count / len(train_components)
    print(f"  Component {k}: {count} samples ({pct:.1f}%)")

print(f"\nDoes the GMM converge? {gmm.converged_}\nnumber of iterations: {gmm.n_iter_}")

Fitted GMM with 3 components

Component sizes in training set:
  Component 0: 4001 samples (50.5%)
  Component 1: 2331 samples (29.4%)
  Component 2: 1597 samples (20.1%)

Does the GMM converge? True
number of iterations: 4


## Step 3: Compute Component-Level Claim Probabilities

For each component, calculate the empirical claim rate (P(Y=1|component=k)) based on training data.

In [7]:
# Calculate claim probability for each component
y_train_array = y_train.values 
component_claim_probs_hard = np.zeros(n_components)
component_claim_probs_soft = np.zeros(n_components)

print("Component Risk Profiles:")
print("-" * 60)
for k in range(n_components):
    # Hard assignments
    mask_hard = (train_components == k)
    n_samples_hard = mask_hard.sum()  

    if n_samples_hard > 0:
        claim_rate_hard = y_train_array[mask_hard].mean()
        n_claims_hard = y_train_array[mask_hard].sum()
    else:
        claim_rate_hard = np.nan
        n_claims_hard = 0
    
    component_claim_probs_hard[k] = claim_rate_hard

    # Soft assignments
    w_k = train_component_probs[:, k]  # Weights for component k
    total_weight = w_k.sum()
    
    if total_weight > 0:
        claim_rate_soft = np.sum(w_k * y_train_array) / total_weight
        effective_claims = np.sum(w_k * y_train_array)
    else:
        claim_rate_soft = np.nan
        effective_claims = 0.0
    
    component_claim_probs_soft[k] = claim_rate_soft
    
    print(f"Component {k}:")
    
    print(f"  HARD:")
    print(f"    Size: {n_samples_hard} samples ({100*n_samples_hard/len(y_train):.1f}%)")
    print(f"    Claims: {n_claims_hard} ({100*claim_rate_hard:.1f}%)")
    print(f"    Risk Level (hard): "f"{'HIGH' if claim_rate_hard > 0.3 else 'MEDIUM' if claim_rate_hard > 0.2 else 'LOW'}")

    print(f"  SOFT (responsibility-weighted):")
    print(f"    Effective size: {total_weight:.1f}")
    print(f"    Effective claims: {effective_claims:.1f}")
    print(f"    Claim rate (soft): {100*claim_rate_soft:.1f}%")
    print(f"    Risk Level (soft): "
          f"{'HIGH' if claim_rate_soft > 0.3 else 'MEDIUM' if claim_rate_soft > 0.2 else 'LOW'}\n")


print(f"Overall training claim rate: {y_train.mean():.4f}")

Component Risk Profiles:
------------------------------------------------------------
Component 0:
  HARD:
    Size: 4001 samples (50.5%)
    Claims: 1955.0 (48.9%)
    Risk Level (hard): HIGH
  SOFT (responsibility-weighted):
    Effective size: 4001.0
    Effective claims: 1955.0
    Claim rate (soft): 48.9%
    Risk Level (soft): HIGH

Component 1:
  HARD:
    Size: 2331 samples (29.4%)
    Claims: 358.0 (15.4%)
    Risk Level (hard): LOW
  SOFT (responsibility-weighted):
    Effective size: 2331.0
    Effective claims: 358.0
    Claim rate (soft): 15.4%
    Risk Level (soft): LOW

Component 2:
  HARD:
    Size: 1597 samples (20.1%)
    Claims: 159.0 (10.0%)
    Risk Level (hard): LOW
  SOFT (responsibility-weighted):
    Effective size: 1597.0
    Effective claims: 159.0
    Claim rate (soft): 10.0%
    Risk Level (soft): LOW

Overall training claim rate: 0.3118


## Step 4: Make Predictions Using Mixture Model

For each customer, we compute:
- P(component=k|X) from the GMM (posterior probabilities)
- P(Y=1|component=k) from training data (component claim rates)
- Overall prediction: P(Y=1|X) = Σ P(component=k|X) × P(Y=1|component=k)

In [8]:
def predict_mixture_model(X_scaled, gmm, component_claim_probs):
    """
    Predict claim probability using mixture model approach.
    
    Args:
        X_scaled: Standardized feature matrix
        gmm: Fitted GaussianMixture model
        component_claim_probs: Array of P(Y=1|component=k) for each component
    
    Returns:
        Array of predicted probabilities P(Y=1|X)
    """
    # Get posterior probabilities P(component=k|X)
    component_posteriors = gmm.predict_proba(X_scaled)
    
    # Compute weighted average: P(Y=1|X) = Σ P(component=k|X) * P(Y=1|component=k)
    y_pred_proba = component_posteriors @ component_claim_probs
    
    return y_pred_proba

# Generate predictions for training set (for sanity check)
y_train_pred_proba = predict_mixture_model(X_train_scaled, gmm, component_claim_probs_soft)
y_train_pred = (y_train_pred_proba >= 0.3).astype(int)

# Generate predictions for test set
y_test_pred_proba = predict_mixture_model(X_test_scaled, gmm, component_claim_probs_hard)
y_test_pred = (y_test_pred_proba >= 0.3).astype(int)

print("Predictions generated successfully!")
print(f"Train predicted claim rate: {y_train_pred.mean():.3f} (actual: {y_train.mean():.4f})")
print(f"Test predicted claim rate: {y_test_pred.mean():.3f} (actual: {y_test.mean():.4f})")

Predictions generated successfully!
Train predicted claim rate: 0.505 (actual: 0.3118)
Test predicted claim rate: 0.520 (actual: 0.3182)


In [ ]:
def analyze_soft_assignments(component_probs, tol=1e-6, n_show=20):
    """
    Analyze non-hard (soft/mixed) component assignments.

    Args:
        component_probs: array of shape (n_samples, n_components),
                         typically from gmm.predict_proba(X)
        tol: tolerance for treating a max probability as "1"
        n_show: how many soft examples to print

    Returns:
        soft_df: DataFrame with soft assignments and sample indices
    """
    probs = np.asarray(component_probs)
    
    max_probs = probs.max(axis=1)
    is_hard = np.isclose(max_probs, 1.0, atol=tol)
    is_soft = ~is_hard

    soft_indices = np.where(is_soft)[0]
    soft_matrix = probs[is_soft]

    print(f"Total samples: {probs.shape[0]}")
    print(f"Hard-like assignments: {is_hard.sum()}")
    print(f"Non-hard (soft/mixed) assignments: {is_soft.sum()}")

    soft_df = pd.DataFrame(
        soft_matrix,
        columns=[f"component_{k}" for k in range(probs.shape[1])]
    )
    soft_df["sample_index"] = soft_indices

    print("\nExamples of non-hard assignments:")
    print(soft_df.head(n_show))

    return soft_df

soft_df = analyze_soft_assignments(train_component_probs, tol=1e-6, n_show=20)


## Step 5: Evaluate Model Performance

Let's assess how well our mixture model predicts insurance claims on the test set.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate metrics
print("=" * 70)
print("MIXTURE MODEL PERFORMANCE ON TEST SET")
print("=" * 70)

# Classification metrics
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
roc_auc = roc_auc_score(y_test, y_test_pred_proba)

print(f"\nAccuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")

# Confusion Matrix
print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)
cm = confusion_matrix(y_test, y_test_pred)
print(f"\n                 Predicted: No Claim  |  Predicted: Claim")
print(f"Actual: No Claim      {cm[0,0]:6d}         |      {cm[0,1]:6d}")
print(f"Actual: Claim         {cm[1,0]:6d}         |      {cm[1,1]:6d}")

# Classification Report
print("\n" + "=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_test, y_test_pred, target_names=['No Claim', 'Claim']))

### Visualize ROC Curve

In [ ]:
# Plot ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Mixture Model', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Analyze Component Assignments on Test Set

In [ ]:
# Analyze how test samples are distributed across components
test_components = gmm.predict(X_test_scaled)
test_component_probs = gmm.predict_proba(X_test_scaled)

print("=" * 70)
print("TEST SET COMPONENT ANALYSIS")
print("=" * 70)

y_test_array = y_test.values

for k in range(n_components):
    mask = (test_components == k)
    n_samples = mask.sum()
    actual_claim_rate = y_test_array[mask].mean()
    predicted_claim_rate = component_claim_probs_hard[k]
    
    print(f"\nComponent {k}:")
    print(f"  Test samples: {n_samples} ({100*n_samples/len(y_test):.1f}%)")
    print(f"  Predicted claim prob (from train): {predicted_claim_rate:.3f}")
    print(f"  Actual claim rate (in test): {actual_claim_rate:.3f}")
    print(f"  Difference: {abs(actual_claim_rate - predicted_claim_rate):.3f}")
    
    # Show prediction confidence
    avg_confidence = test_component_probs[mask, k].mean()
    print(f"  Average membership confidence: {avg_confidence:.3f}")

### Visualize Predicted vs Actual Probabilities

In [ ]:
# Distribution of predicted probabilities by actual outcome
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of predicted probabilities
axes[0].hist(y_test_pred_proba[y_test == 0], bins=30, alpha=0.6, label='No Claim (Actual)', color='blue', edgecolor='black')
axes[0].hist(y_test_pred_proba[y_test == 1], bins=30, alpha=0.6, label='Claim (Actual)', color='red', edgecolor='black')
axes[0].axvline(0.5, color='green', linestyle='--', linewidth=2, label='Decision Threshold')
axes[0].set_xlabel('Predicted Probability of Claim', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Predicted Probabilities', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot by actual outcome
test_df = pd.DataFrame({
    'Predicted_Prob': y_test_pred_proba,
    'Actual_Outcome': y_test
})
sns.boxplot(x='Actual_Outcome', y='Predicted_Prob', data=test_df, ax=axes[1])
axes[1].set_xlabel('Actual Outcome', fontsize=12)
axes[1].set_ylabel('Predicted Probability', fontsize=12)
axes[1].set_title('Predicted Probabilities by Actual Outcome', fontsize=13, fontweight='bold')
axes[1].set_xticklabels(['No Claim', 'Claim'])
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()